Embeddings Generator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q sentence-transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.1 MB/s eta 0:00:00


In [ ]:
import os
os.environ["USE_TF"] = "0" # Keep this to ensure PyTorch is used

from sentence_transformers import SentenceTransformer
import json, numpy as np
import torch # Import torch to check device

# --- Configuration ---
# IMPORTANT: Update this path to where you uploaded the file in Google Drive
# Example assumes you created a 'Colab_Data' folder in your Drive's root
data_file_path = '/content/drive/MyDrive/COS484/openmathinstruct_processed_15k.jsonl'

# Decide where to save the output embeddings file in Google Drive
output_path_drive = "/content/drive/MyDrive/COS484/openmathinstruct_user_embeds.npy"
# --- End Configuration ---

# Verify GPU is available (optional but good practice)
if torch.cuda.is_available():
    device = 'cuda'
    print("GPU is available. Using CUDA.")
else:
    # Fallback to CPU if GPU somehow isn't available after setting runtime
    # This shouldn't happen if you correctly set the runtime, but good to check
    device = 'cpu'
    print("GPU not available. Using CPU (This will be very slow).")

    # Check if data file exists before proceeding
if not os.path.exists(data_file_path):
    print(f"ERROR: Data file not found at: {data_file_path}")
    print("Please ensure you uploaded the file to Google Drive and the path is correct.")


GPU is available. Using CUDA.


In [ ]:
# Check if data file exists before proceeding
if not os.path.exists(data_file_path):
    print(f"ERROR: Data file not found at: {data_file_path}")
    print("Please ensure you uploaded the file to Google Drive and the path is correct.")
else:
    print(f"Data file found at: {data_file_path}")

    # Load the embedding model - let it auto-detect the device (should be 'cuda')
    print("Loading the embedding model...")
    # We explicitly pass the detected device for clarity, though auto-detection should work
    model = SentenceTransformer("intfloat/e5-large-v2", device=device)
    print("Model loaded.")

    # Process the OpenMathInstruct dataset
    print("Processing dataset...")
    texts = []
    try:
        with open(data_file_path, "r") as f:
            for i, line in enumerate(f):
                try:
                    example = json.loads(line)
                    # Extract the user message (the instruction)
                    user_message = example["messages"][0]["content"]
                    texts.append(user_message)
                except (json.JSONDecodeError, KeyError, IndexError) as e:
                    print(f"Warning: Skipping line {i+1} due to error: {e}")
                    # Optionally: print(f"Problematic line: {line.strip()}")
                    continue # Skip to the next line

        print(f"Successfully loaded {len(texts)} texts.")

        # Encode all user questions (not the answers!)
        print("Starting encoding...")
        embeddings = model.encode(
            texts,
            batch_size=32,  # START WITH A SMALLER BATCH SIZE ON COLAB T4 GPUs
            normalize_embeddings=True,
            show_progress_bar=True
        )
        print("Encoding complete.")

        # Save for downstream selection directly to Google Drive
        print(f"Saving embeddings to: {output_path_drive}")
        np.save(output_path_drive, embeddings)
        print("Embeddings saved successfully to Google Drive.")

    except Exception as e:
        print(f"An error occurred during processing: {e}")

Data file found at: /content/drive/MyDrive/COS484/openmathinstruct_processed_15k.jsonl
Loading the embedding model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Model loaded.
Processing dataset...
Successfully loaded 15000 texts.
Starting encoding...


Batches:   0%|          | 0/469 [00:00<?, ?it/s]

Encoding complete.
Saving embeddings to: /content/drive/MyDrive/COS484/openmathinstruct_user_embeds.npy
Embeddings saved successfully to Google Drive.


Apricot

In [ ]:
# Make sure apricot is installed alongside other dependencies
!pip install -q sentence-transformers torch apricot-select

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 14.2 MB/s eta 0:00:00


In [ ]:
# Make sure apricot is installed alongside other dependencies
# !pip install -q sentence-transformers torch apricot-select # Run this if needed

import numpy as np
from apricot import FacilityLocationSelection
import os # To check file paths
import json # Needed if you want to load the selected data later
import time # To add timestamp if desired

# --- Configuration ---

# !! IMPORTANT !! Make sure this path matches EXACTLY where your embeddings were saved
embeddings_file_path = "/content/drive/MyDrive/COS484/openmathinstruct_user_embeds.npy" # Or openmath15k_e5.npy, adjust as needed

# Path to your *original* data file (needed to retrieve the selected text)
original_data_file_path = '/content/drive/MyDrive/COS484/openmathinstruct_processed_15k.jsonl'

# --- Parameters for Distinct Naming ---
# Base name related to the source embeddings (helps link outputs)
BASE_NAME = "openmath15k_e5" # Or adjust if your embedding file name basis is different

# Tag for the specific selection method used
# Using 'apricot_fl' for Apricot Facility Location
SELECTION_METHOD_TAG = "apricot_fl"

# Number of data points you want to select
NUM_SAMPLES_TO_SELECT = 1000

# Optional: Add a timestamp for uniqueness if running multiple times
# timestamp = time.strftime("%Y%m%d-%H%M%S")
# --- End Parameters ---


# --- Construct Descriptive Output Filenames ---
# Example: /content/drive/MyDrive/COS484/openmath15k_e5_apricot_fl_1000_indices.npy
output_indices_path = f"/content/drive/MyDrive/COS484/{BASE_NAME}_{SELECTION_METHOD_TAG}_{NUM_SAMPLES_TO_SELECT}_indices.npy"
# Example: /content/drive/MyDrive/COS484/openmath15k_e5_apricot_fl_1000_data.jsonl
selected_data_output_path = f"/content/drive/MyDrive/COS484/{BASE_NAME}_{SELECTION_METHOD_TAG}_{NUM_SAMPLES_TO_SELECT}_data.jsonl"

# --- End Configuration ---


print(f"--- Starting Data Selection using: {SELECTION_METHOD_TAG} ---")
print(f"Input Embeddings: {embeddings_file_path}")
print(f"Output Indices File: {output_indices_path}")
print(f"Output Data File: {selected_data_output_path}")


# 1. Check if the embeddings file exists
if not os.path.exists(embeddings_file_path):
    print(f"ERROR: Embeddings file not found at: {embeddings_file_path}")
    print("Please ensure the embedding generation script ran successfully and saved the file with this exact name.")
else:
    print(f"\nLoading embeddings...")
    try:
        # 2. Load the embeddings
        X = np.load(embeddings_file_path)
        print(f"Embeddings loaded. Shape: {X.shape}")

        # Sanity check: ensure we have enough data points
        if X.shape[0] < NUM_SAMPLES_TO_SELECT:
             print(f"Warning: Requested {NUM_SAMPLES_TO_SELECT} samples, but only {X.shape[0]} embeddings available.")
             print(f"Adjusting selection count to {X.shape[0]}.")
             NUM_SAMPLES_TO_SELECT = X.shape[0]
        elif X.shape[0] == 0:
             raise ValueError("Embeddings file seems to be empty.")
        elif NUM_SAMPLES_TO_SELECT <= 0:
             raise ValueError("NUM_SAMPLES_TO_SELECT must be greater than 0.")

        # 3. Initialize the Apricot Facility Location selector
        print(f"\nInitializing FacilityLocationSelection to select {NUM_SAMPLES_TO_SELECT} samples...")
        selector = FacilityLocationSelection(
            n_samples=NUM_SAMPLES_TO_SELECT,
            metric='cosine', # Good for normalized sentence embeddings
            optimizer='lazy' # Efficient greedy optimizer
        )

        # 4. Fit the selector to the embeddings data
        print("Fitting the selector (this might take a moment)...")
        selector.fit(X)
        print("Selector fitting complete.")

        # 5. Get the indices of the selected items
        chosen_indices = selector.ranking[:NUM_SAMPLES_TO_SELECT]
        print(f"\nSelected {len(chosen_indices)} indices using {SELECTION_METHOD_TAG}.")

        # 6. Save the selected indices with the descriptive name
        print(f"Saving selected indices to: {output_indices_path}")
        np.save(output_indices_path, chosen_indices)
        print("Selected indices saved successfully.")

        # --- Optional but Recommended: Retrieve and save the actual data for the selected indices ---
        print("\n--- Loading corresponding data for selected indices ---")
        if not os.path.exists(original_data_file_path):
            print(f"Warning: Original data file not found at {original_data_file_path}. Skipping data retrieval.")
        else:
            selected_data = []
            print(f"Reading original data from {original_data_file_path}...")
            try:
                # Read all lines first
                with open(original_data_file_path, "r") as f:
                    original_lines = f.readlines()

                print(f"Retrieving data for {len(chosen_indices)} selected indices...")
                for idx in chosen_indices:
                    if 0 <= idx < len(original_lines):
                        try:
                            selected_data.append(json.loads(original_lines[idx]))
                        except json.JSONDecodeError:
                            print(f"Warning: Could not decode JSON on line {idx+1} of original file.")
                    else:
                        print(f"Warning: Index {idx} is out of bounds for the original data file (length {len(original_lines)}).")

                print(f"Successfully retrieved {len(selected_data)} data points selected via {SELECTION_METHOD_TAG}.")

                # Save the selected data subset to the new descriptive JSONL file
                if selected_data:
                    print(f"Saving selected data subset to: {selected_data_output_path}")
                    with open(selected_data_output_path, 'w') as outfile:
                        for entry in selected_data:
                            json.dump(entry, outfile)
                            outfile.write('\n')
                    print("Selected data saved successfully.")

            except Exception as e:
                print(f"An error occurred during data retrieval/saving: {e}")


    except Exception as e:
        print(f"\nAn error occurred during the Apricot selection process: {e}")

print(f"\n--- {SELECTION_METHOD_TAG} Selection Cell Complete ---")

--- Starting Data Selection using: apricot_fl ---
Input Embeddings: /content/drive/MyDrive/COS484/openmathinstruct_user_embeds.npy
Output Indices File: /content/drive/MyDrive/COS484/openmath15k_e5_apricot_fl_1000_indices.npy
Output Data File: /content/drive/MyDrive/COS484/openmath15k_e5_apricot_fl_1000_data.jsonl

Loading embeddings...
Embeddings loaded. Shape: (15000, 1024)

Initializing FacilityLocationSelection to select 1000 samples...
Fitting the selector (this might take a moment)...
Selector fitting complete.

Selected 1000 indices using apricot_fl.
Saving selected indices to: /content/drive/MyDrive/COS484/openmath15k_e5_apricot_fl_1000_indices.npy
Selected indices saved successfully.

--- Loading corresponding data for selected indices ---
Reading original data from /content/drive/MyDrive/COS484/openmathinstruct_processed_15k.jsonl...
Retrieving data for 1000 selected indices...
Successfully retrieved 1000 data points selected via apricot_fl.
Saving selected data subset to: /co

Max Distance

In [ ]:
import numpy as np
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm # Use notebook version for Colab progress bar
import time

# --- Configuration ---

# !! IMPORTANT !! Path to your saved embeddings file
embeddings_file_path = "/content/drive/MyDrive/COS484/openmathinstruct_user_embeds.npy" # Adjust as needed

# Path to your *original* data file (needed to retrieve the selected text)
original_data_file_path = '/content/drive/MyDrive/COS484/openmathinstruct_processed_15k.jsonl'

# Base name related to the source embeddings (helps link outputs)
BASE_NAME = "openmath15k_e5" # Or adjust if your embedding file name basis is different

# Tag for this specific selection method
SELECTION_METHOD_TAG = "mdiv"

# Number of data points you want to select
NUM_SAMPLES_TO_SELECT = 1000

# --- End Configuration ---


# --- Construct Descriptive Output Filenames ---
output_indices_path = f"/content/drive/MyDrive/COS484/{BASE_NAME}_{SELECTION_METHOD_TAG}_{NUM_SAMPLES_TO_SELECT}_indices.npy"
selected_data_output_path = f"/content/drive/MyDrive/COS484/{BASE_NAME}_{SELECTION_METHOD_TAG}_{NUM_SAMPLES_TO_SELECT}_data.jsonl"
# --- End Filename Construction ---


print(f"--- Starting Data Selection using: {SELECTION_METHOD_TAG} (Maximize Diversity) ---")
print(f"Input Embeddings: {embeddings_file_path}")
print(f"Output Indices File: {output_indices_path}")
print(f"Output Data File: {selected_data_output_path}")
print(f"Number of samples to select: {NUM_SAMPLES_TO_SELECT}")


# 1. Check if the embeddings file exists
if not os.path.exists(embeddings_file_path):
    print(f"ERROR: Embeddings file not found at: {embeddings_file_path}")
    print("Please ensure the embedding generation script ran successfully and saved the file.")
else:
    print(f"\nLoading embeddings...")
    try:
        # 2. Load the embeddings
        X = np.load(embeddings_file_path).astype(np.float32) # Load as float32 for efficiency
        print(f"Embeddings loaded. Shape: {X.shape}")
        num_total_samples = X.shape[0]

        # 3. Check requested sample size
        if NUM_SAMPLES_TO_SELECT > num_total_samples:
             print(f"Warning: Requested {NUM_SAMPLES_TO_SELECT}, but only {num_total_samples} embeddings available.")
             print(f"Adjusting selection count to {num_total_samples}.")
             NUM_SAMPLES_TO_SELECT = num_total_samples
        elif NUM_SAMPLES_TO_SELECT <= 0:
             raise ValueError("NUM_SAMPLES_TO_SELECT must be greater than 0.")
        elif num_total_samples == 0:
             raise ValueError("Embeddings file seems to be empty.")

        # 4. Normalize embeddings (important for cosine similarity/distance)
        print("Normalizing embeddings (L2 norm)...")
        norms = np.linalg.norm(X, axis=1, keepdims=True)
        # Avoid division by zero for potential zero vectors
        norms[norms == 0] = 1e-10
        X = X / norms

        # --- mdiv Selection Logic ---
        print("\nStarting mdiv selection process...")
        selected_indices = set()
        all_indices = set(range(num_total_samples))

        # Select the first point: the one furthest from the centroid
        print("Calculating centroid and selecting first point (max distance from mean)...")
        if num_total_samples > 0:
            centroid = np.mean(X, axis=0, keepdims=True)
            # Similarity shape: (num_total_samples, 1)
            sim_to_centroid = cosine_similarity(X, centroid).flatten()
            first_index = np.argmin(sim_to_centroid) # Index with min similarity = max distance
            selected_indices.add(first_index)
            remaining_indices = all_indices - selected_indices
            print(f"Selected first index: {first_index}")
        else:
             raise ValueError("Cannot select from empty embeddings.")

        # Iteratively select the rest
        if NUM_SAMPLES_TO_SELECT > 1:
            print(f"Starting iterative selection for remaining {NUM_SAMPLES_TO_SELECT - 1} points...")
            pbar = tqdm(total=NUM_SAMPLES_TO_SELECT - 1, desc="Selecting diverse points")

            while len(selected_indices) < NUM_SAMPLES_TO_SELECT:
                if not remaining_indices:
                    print("Warning: No remaining indices to select from.")
                    break

                start_iter_time = time.time()

                # Convert sets to lists/arrays for indexing embeddings
                list_remaining = list(remaining_indices)
                list_selected = list(selected_indices)

                # Calculate similarities between each remaining point and all selected points
                # Shape: (n_remaining, n_selected)
                # This can be the slowest step
                sim_matrix_subset = cosine_similarity(X[list_remaining, :], X[list_selected, :])

                # Sum similarities for each remaining point (sum across columns)
                # Lower sum means more distant overall from the selected set
                # Shape: (n_remaining,)
                sum_sim_per_remaining = np.sum(sim_matrix_subset, axis=1)

                # Find the index within list_remaining that corresponds to the minimum sum similarity
                min_sum_sim_idx_in_list = np.argmin(sum_sim_per_remaining)

                # Get the original data index of the best candidate
                best_original_index = list_remaining[min_sum_sim_idx_in_list]

                # Add the best candidate to selected set and remove from remaining set
                selected_indices.add(best_original_index)
                remaining_indices.remove(best_original_index)

                pbar.update(1)
                iter_time = time.time() - start_iter_time
                # Optional: Postfix to show progress and iteration time
                pbar.set_postfix_str(f"{len(selected_indices)} selected, last iter: {iter_time:.2f}s")

            pbar.close()
        # --- End mdiv Selection Logic ---

        print(f"\nFinished selection. Total selected indices: {len(selected_indices)}")

        # 7. Save the selected indices
        final_indices_array = np.array(list(selected_indices), dtype=int)
        # Sort indices for potentially easier data retrieval later (optional)
        # final_indices_array.sort()
        print(f"Saving {len(final_indices_array)} selected indices to: {output_indices_path}")
        np.save(output_indices_path, final_indices_array)
        print("Selected indices saved successfully.")


        # --- Optional: Retrieve and save the actual data for the selected indices ---
        print("\n--- Loading corresponding data for selected indices ---")
        if not os.path.exists(original_data_file_path):
            print(f"Warning: Original data file not found at {original_data_file_path}. Skipping data retrieval.")
        else:
            selected_data = []
            print(f"Reading original data from {original_data_file_path}...")
            try:
                # Read all lines first
                with open(original_data_file_path, "r") as f:
                    # Use list comprehension for potential memory efficiency over readlines()
                    original_lines = [line for line in f]

                print(f"Retrieving data for {len(final_indices_array)} selected indices...")
                retrieved_count = 0
                for idx in final_indices_array: # Iterate through the selected indices
                    if 0 <= idx < len(original_lines):
                        try:
                            selected_data.append(json.loads(original_lines[idx]))
                            retrieved_count += 1
                        except json.JSONDecodeError:
                            print(f"Warning: Could not decode JSON on line {idx+1} of original file.")
                    else:
                        print(f"Warning: Index {idx} is out of bounds for the original data file (length {len(original_lines)}).")

                print(f"Successfully retrieved {retrieved_count} data points selected via {SELECTION_METHOD_TAG}.")

                # Save the selected data subset to the new descriptive JSONL file
                if selected_data:
                    print(f"Saving selected data subset to: {selected_data_output_path}")
                    with open(selected_data_output_path, 'w') as outfile:
                        for entry in selected_data:
                            json.dump(entry, outfile)
                            outfile.write('\n')
                    print("Selected data saved successfully.")

            except MemoryError:
                 print("\nERROR: Ran out of memory while trying to load the original JSONL file.")
                 print("Consider processing the JSONL file line-by-line if it's too large to fit in RAM.")
            except Exception as e:
                print(f"An error occurred during data retrieval/saving: {e}")

    except MemoryError:
        print("\nERROR: A MemoryError occurred, likely during embedding loading or similarity calculation.")
        print("Consider using a Colab runtime with more RAM (Colab Pro) or reducing the dataset size if feasible.")
    except Exception as e:
        print(f"\nAn error occurred during the {SELECTION_METHOD_TAG} selection process: {e}")

print(f"\n--- {SELECTION_METHOD_TAG} Selection Cell Complete ---")

--- Starting Data Selection using: mdiv (Maximize Diversity) ---
Input Embeddings: /content/drive/MyDrive/COS484/openmathinstruct_user_embeds.npy
Output Indices File: /content/drive/MyDrive/COS484/openmath15k_e5_mdiv_1000_indices.npy
Output Data File: /content/drive/MyDrive/COS484/openmath15k_e5_mdiv_1000_data.jsonl
Number of samples to select: 1000

Loading embeddings...
Embeddings loaded. Shape: (15000, 1024)
Normalizing embeddings (L2 norm)...

Starting mdiv selection process...
Calculating centroid and selecting first point (max distance from mean)...
Selected first index: 1310
Starting iterative selection for remaining 999 points...


Selecting diverse points:   0%|          | 0/999 [00:00<?, ?it/s]


Finished selection. Total selected indices: 1000
Saving 1000 selected indices to: /content/drive/MyDrive/COS484/openmath15k_e5_mdiv_1000_indices.npy
Selected indices saved successfully.

--- Loading corresponding data for selected indices ---
Reading original data from /content/drive/MyDrive/COS484/openmathinstruct_processed_15k.jsonl...
Retrieving data for 1000 selected indices...
Successfully retrieved 1000 data points selected via mdiv.
Saving selected data subset to: /content/drive/MyDrive/COS484/openmath15k_e5_mdiv_1000_data.jsonl
Selected data saved successfully.

--- mdiv Selection Cell Complete ---


An End-to-End Submodular Framework for Data-Efficient In-Context
Learning